In [25]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import pandas as pd
import pickle
import random
import tempfile


In [26]:
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)


In [27]:
words = []
classes = []
documents = []
ignore_words = []

for intent in intents ['intents']:
    for pattern in intent ['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

            words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
            words = sorted(list(set(words)))
            classes = sorted(list(set(classes)))
            print(len(documents), "documents")
            print (len(classes), "classes", classes)
            print (len(words), "unique stemmed words", words)

            pickle.dump(words, open('words.pkl', 'wb'))
            pickle.dump(classes, open('classes.pkl', 'wb'))




1 documents
1 classes ['unknown']
1 unique stemmed words ['jbjkbk']
7 documents
2 classes ['greeting', 'unknown']
6 unique stemmed words ['126889', 'hi', 'jbjkbk', 'kdd', 'kjbh', 'ljhh']
13 documents
3 classes ['goodbye', 'greeting', 'unknown']
19 unique stemmed words ['126889', '?', 'anyon', 'ar', 'cya', 'day', 'good', 'hello', 'hi', 'how', 'is', 'jbjkbk', 'kdd', 'kjbh', 'ljhh', 'ther', 'up', 'what', 'you']
18 documents
4 classes ['age', 'goodbye', 'greeting', 'unknown']
28 unique stemmed words ['126889', '?', 'a', 'am', 'anyon', 'ar', 'cya', 'day', 'good', 'goodby', 'hav', 'hello', 'hi', 'how', 'i', 'is', 'jbjkbk', 'kdd', 'kjbh', 'lat', 'leav', 'ljhh', 'old', 'see', 'ther', 'up', 'what', 'you']
23 documents
5 classes ['age', 'goodbye', 'greeting', 'name', 'unknown']
32 unique stemmed words ['126889', '?', 'a', 'ag', 'am', 'anyon', 'ar', 'cya', 'day', 'good', 'goodby', 'hav', 'hello', 'hi', 'how', 'i', 'is', 'jbjkbk', 'kdd', 'kjbh', 'lat', 'leav', 'ljhh', 'nam', 'old', 'see', 'ther', 

In [28]:
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]

    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

        output_row = list(output_empty)
        output_row[classes.index(doc[1])] = 1

        training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])
print ("Training data created")



Training data created


In [29]:
model = keras.Sequential()
model.add(keras.layers.Dense(128, input_shape=(len (train_x[0]),), activation ='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(len(train_y[0]), activation='softmax'))

In [30]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               12800     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 22)                1430      
Total params: 22,486
Trainable params: 22,486
Non-trainable params: 0
_________________________________________________________________


In [31]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [32]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=50, batch_size=5, verbose=1)

Epoch 1/50
2555/2555 [==============================] - 8s 3ms/step - loss: 0.5517 - accuracy: 0.8408
Epoch 2/50
2555/2555 [==============================] - 8s 3ms/step - loss: 0.0910 - accuracy: 0.9720
Epoch 3/50
2555/2555 [==============================] - 8s 3ms/step - loss: 0.0580 - accuracy: 0.9801
Epoch 4/50
2555/2555 [==============================] - 8s 3ms/step - loss: 0.0516 - accuracy: 0.9816
Epoch 5/50
2555/2555 [==============================] - 9s 3ms/step - loss: 0.0424 - accuracy: 0.9848
Epoch 6/50
2555/2555 [==============================] - 8s 3ms/step - loss: 0.0413 - accuracy: 0.9839
Epoch 7/50
2555/2555 [==============================] - 8s 3ms/step - loss: 0.0322 - accuracy: 0.9876
Epoch 8/50
2555/2555 [==============================] - 8s 3ms/step - loss: 0.0369 - accuracy: 0.9848
Epoch 9/50
2555/2555 [==============================] - 8s 3ms/step - loss: 0.0360 - accuracy: 0.9856
Epoch 10/50
2555/2555 [==============================] - 8s 3ms/step - loss: 0.031

In [33]:
def clean_up_sentence (sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:

                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))
            

In [34]:
p = bow("How are you today", words)
print (p)
print (classes)

found in bag: how
found in bag: ar
found in bag: you
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
['Hate', 'None', 'age', 'banter', 'body-pain', 'breathing', 'cough', 'diarrhoea', 'fever', 'goodbye', 'greeting', 'headache', 'name', 'nasal', 'noanswer', 'options', 'sore-throat', 'thanks', 'tiredness', 'unknown', 'vomit', 'yes']


In [35]:
inputvar = pd.DataFrame([p], dtype=float, index=['input'])
print(model.predict(inputvar))

[[1.0356118e-11 2.8539363e-21 4.4036930e-10 8.7628099e-12 7.9814322e-15
  4.0348631e-14 2.3085668e-17 6.5376065e-15 3.3665204e-13 4.7858236e-12
  1.0000000e+00 3.0534569e-14 4.2115007e-14 1.1437704e-16 1.7113509e-11
  2.6113933e-09 4.4175755e-13 1.1810736e-11 3.5349046e-14 6.9500142e-11
  2.1666576e-15 7.5868201e-12]]


In [36]:
 model.save("chappy-bot-model.h5", hist)



In [37]:


pickle.dump({'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open ( "chappy-bot-data.pkl", "wb"))

